In [123]:
from ingestion import ingestion
import pandas as pd

In [124]:
path = "./data/"

In [125]:
train  = ingestion.ingest(path)

['test_cases.zip', 'test_cases1.zip']
Select the file(zero indexed): 1
The File contains: 891 rows and 12 columns


In [126]:
from seperation import seperation
idx, categorical, continous = seperation.seperation(train)

In [127]:
from imputation import imputation

In [128]:
train, imputed_with = imputation.imputation(train)

(['PassengerId'], ['Survived', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], ['Age', 'Fare'])
Your selected dataframe has 12 columns.
There are 3 columns that have missing values.
          Missing Values  % of Total Values
Cabin                687               77.1
Age                  177               19.9
Embarked               2                0.2


In [129]:
imputed_with

{'Age': 28.0, 'Cabin': 'NA', 'Embarked': 'NA'}

In [130]:
continous

['Age', 'Fare']

In [131]:
def feature_generation_cont(df, continous, target):
    """Continous Variables"""
    
    for i in range(len(continous)-1):
        var_1 = continous[i]
        var_2 = continous[i+1]
        # ratios
        df["ratio_"+str(var_1)+"_"+str(var_2)] = df[var_1].astype("float")/df[var_2].astype("float")
        #products
        df["prod_"+str(var_1)+"_"+str(var_2)] = df[var_1].astype("float")*df[var_2].astype("float")
    
    for i in range(len(continous)):
        var_1 = continous[i]
        #square
        df["sqr_"+str(var_1)] = df[var_1].apply(lambda x: x**2)
        
    return df
    

In [132]:
train = feature_generation_cont(train, continous, "Survived")

In [133]:
train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked',
       u'Cabin_imputed', u'Age_imputed', u'Embarked_imputed',
       u'ratio_Age_Fare', u'prod_Age_Fare', u'sqr_Age', u'sqr_Fare'],
      dtype='object')

In [134]:
def count_to_dict(df, col):
    grp = df.groupby([col])[col].agg(['count']).reset_index()
    dt = {}
    for i in range(grp.shape[0]):
        dt[grp.iloc[i,0]] = grp.iloc[i,1]
    return dt

In [135]:
import pandas as pd

In [136]:
def feature_generation_catg(df, categorical, target):
    '''categorical variables'''
    for col in categorical:
        if col == target:
            pass
        else:
            dt = count_to_dict(df, col)
            df["count_"+str(col)] = df[col].apply(lambda x: dt[x])
    
    one_hot = []
    drop = []
    for col in categorical:
        if col == target:
            pass
        else:
            if float(len(df[col].unique()))/df.shape[0] <= .10:
                one_hot.append(col)
            else:
                drop.append(col)
    df = pd.get_dummies(df, columns = one_hot, drop_first = True)
    df = df.drop(drop, axis = 1)
    
                
    return df

In [137]:
train = feature_generation_catg(train, categorical, "Survived")
train.columns

Index([u'PassengerId', u'Survived', u'Age', u'Fare', u'Cabin_imputed',
       u'Age_imputed', u'Embarked_imputed', u'ratio_Age_Fare',
       u'prod_Age_Fare', u'sqr_Age', u'sqr_Fare', u'count_Pclass',
       u'count_Name', u'count_Sex', u'count_SibSp', u'count_Parch',
       u'count_Ticket', u'count_Cabin', u'count_Embarked', u'Pclass_2',
       u'Pclass_3', u'Sex_male', u'SibSp_1', u'SibSp_2', u'SibSp_3',
       u'SibSp_4', u'SibSp_5', u'SibSp_8', u'Parch_1', u'Parch_2', u'Parch_3',
       u'Parch_4', u'Parch_5', u'Parch_6', u'Embarked_NA', u'Embarked_Q',
       u'Embarked_S'],
      dtype='object')

In [138]:
train.head(5)

,PassengerId,Survived,Age,Fare,Cabin_imputed,Age_imputed,Embarked_imputed,ratio_Age_Fare,prod_Age_Fare,sqr_Age,...,SibSp_8,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_NA,Embarked_Q,Embarked_S
0,1,0,22.0,7.2500,1,0,0,3.034483,159.5000,484.0,...,0,0,0,0,0,0,0,0,0,1
1,2,1,38.0,71.2833,0,0,0,0.533084,2708.7654,1444.0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,26.0,7.9250,1,0,0,3.280757,206.0500,676.0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,35.0,53.1000,0,0,0,0.659134,1858.5000,1225.0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,35.0,8.0500,1,0,0,4.347826,281.7500,1225.0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
def feature_generation(df, continous, categorical, target):
    df = feature_generation_cont(df, continous, target)
    df = feature_generation_catg(df, categorical, target)